In [0]:
# This is the classifier to determine whether the fetlock-joint has a bone fragment stuck.
import os

%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob
import os
import pickle

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

import cv2

In [0]:
# Load the dataset as pickles

from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

%cd /content/gdrive/My\ Drive/Deep_Learning/
with open("augmented_healthy_data.pickle", 'rb') as handle:
    healthy_samples = pickle.load(handle)

with open("augmented_splinter_data.pickle", 'rb') as handle:
    splinter_samples = pickle.load(handle)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Deep_Learning


In [0]:
def resized_pictures_coord_to_sigmoid(x_y):
  # To avoid predicting x and y outside of the frame. The output of the network will be squeezed between 0-1.
  # In height Pixel 21 = 0 | Pixel 89 = 1
  # In width  Pixel 21 = 0 | Pixel 44 = 1 
  # This is because the input size is 110 x 65. And the crop is 42x42. (21 to from middle to each size)
  
  # new_metric = [0 1];
  # height = [21 89];
  # width = [21 44];
  # coeffs_h = polyfit(height, new_metric, 1);
  # coeffs_w = polyfit(width, new_metric, 1);
  
  return np.round(x_y * [0.0434782608695652, 0.0147058823529412] - [0.913043478260871, 0.308823529411765], 4)

def squeezed_to_coords(x_y_norm):
  # To go from 0 - 1 to the coodinate system
  # h_coeffs = polyfit(new_metric, height, 1);
  # w_coeffs = polyfit(new_metric, width, 1);

  return np.round(x_y_norm * [23, 68] + [21, 21], 0).astype(int)


def rgb2gray(rgb):
  return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def crop_pictures(large_pics, xy_coords):
  n_samples = large_pics.shape[0]
  cropped_array = np.ones(shape=(n_samples, 420, 420, 1)) # 3 if COLOR, 1 if NOT
  for i, data in enumerate(zip(large_pics, xy_coords)):
    pic_l = data[0]
    xy_s = data[1]
    x_rect_start = (xy_s-21)*10
    x_rect_end = (xy_s+21)*10
    cropped_array[i, :, :, :] = pic_l[x_rect_start[1]:x_rect_end[1], x_rect_start[0]:x_rect_end[0]]
  return cropped_array

In [0]:
# HEALTHY
h_pictures_large = []
h_pictures_small = []
h_labels = []
h_xy = []

for data in zip(healthy_samples[:, 0], healthy_samples[:, 1], healthy_samples[:, 3]):
  h_pictures_large.append(rgb2gray(data[0]).reshape(1100, 650, 1))
  h_pictures_small.append(data[1])
  #h_xy.append(data[3])
  #train_labels.append(data[2])
  if data[2] == 0:
    h_labels.append([1, 0])
  if data[2] == 1:
    h_labels.append([0, 1])
  
h_pictures_large = np.array(h_pictures_large)
h_pictures_small = np.array(h_pictures_small)
h_labels = np.array(h_labels)

print("Large Training picture shapes: " + str(h_pictures_large.shape))
print("Small Training picture shapes: " + str(h_pictures_small.shape))
print("Train labels shapes: " + str(h_labels.shape))


Large Training picture shapes: (332, 1100, 650, 1)
Small Training picture shapes: (332, 110, 65, 1)
Train labels shapes: (332, 2)


In [0]:
# SPLINTERED
s_pictures_large = []
s_pictures_small = []
s_labels = []
s_xy = []

for data in zip(splinter_samples[:, 0], splinter_samples[:, 1], splinter_samples[:, 3]):
  s_pictures_large.append(rgb2gray(data[0]).reshape(1100, 650, 1))
  s_pictures_small.append(data[1])
  #h_xy.append(data[3])
  #train_labels.append(data[2])
  if data[2] == 0:
    s_labels.append([1, 0])
  if data[2] == 1:
    s_labels.append([0, 1])
  
s_pictures_large = np.array(s_pictures_large)
s_pictures_small = np.array(s_pictures_small)
s_labels = np.array(s_labels)

print("Large Training picture shapes: " + str(s_pictures_large.shape))
print("Small Training picture shapes: " + str(s_pictures_small.shape))
print("Train labels shapes: " + str(s_labels.shape))


Large Training picture shapes: (324, 1100, 650, 1)
Small Training picture shapes: (324, 110, 65, 1)
Train labels shapes: (324, 2)


In [0]:
num_outputs = 2 # [X Y]
input_channels = 1
batch_size = 1

Localizer = 0
localizer = 0

class Localizer(nn.Module):
    def __init__(self, num_classes, batch_size):
        super(Localizer, self).__init__()
        self.batch_size = batch_size
                
        # Convolutional stuff
        self.simple_conv1 = nn.Conv2d(in_channels=input_channels,
                                     out_channels=32, 
                                     kernel_size=3, 
                                     padding=1,
                                     stride = 2)
        
        self.simple_pool = nn.MaxPool2d(2, 2)
        
        self.simple_conv2 = nn.Conv2d(in_channels = 32,
                                      out_channels= 16,
                                      kernel_size = 3,
                                      padding = 1)
        
        self.simple_conv3 = nn.Conv2d(in_channels = 16,
                                      out_channels= 8,
                                      kernel_size = 3,
                                      padding = 1)

      
        self.linear1 = Linear(in_features=8*6*4,
                            out_features=90)
        self.linear2 = Linear(in_features=90,
                             out_features=30)

        self.l_out = Linear(in_features=30,
                            out_features=num_outputs,
                            bias=False)


        # Activation
        self.activation_relu = torch.nn.ReLU()
        self.activation_sigmoid = torch.nn.Sigmoid()
        
        # Regularization
        self.drop_layer_1 = torch.nn.Dropout(p=0.4)
        self.drop_layer_2 = torch.nn.Dropout(p=0.3)
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.bn2 = torch.nn.BatchNorm2d(16)
        self.bn3 = torch.nn.BatchNorm2d(8)
        

    def forward(self, x):
        # Comes in B H W C (0, 1, 2, 3) comes out B C H W (0, 3, 1, 2)
        x = x.permute(0, 3, 1, 2)

        x = self.simple_conv1(x)
        x = self.bn1(x)
        x = self.activation_relu(x)
        x = self.simple_pool(x)
        
        x = self.simple_conv2(x)
        x = self.bn2(x)
        x = self.activation_relu(x)
        x = self.simple_pool(x)
        
        x = self.simple_conv3(x)
        x = self.bn3(x)
        x = self.activation_relu(x)
        x = self.simple_pool(x)

        x = x.view(-1, 8*6*4) # output dimensions from conv2 to a 1d
        
        x = self.linear1(x)
        x = self.activation_relu(x)
        x = self.drop_layer_1(x)
        
        x = self.linear2(x)
        x = self.activation_relu(x)
        x = self.drop_layer_2(x)
      
        return self.activation_sigmoid(self.l_out(x))
    
localizer = Localizer(num_outputs, batch_size)
print(Localizer)

path = F"/content/gdrive/My Drive/Localizer_warmstarted.pt" 
localizer.load_state_dict(torch.load(path), strict=True)

localizer.float()

<class '__main__.Localizer'>


Localizer(
  (simple_conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (simple_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (simple_conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (simple_conv3): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear1): Linear(in_features=192, out_features=90, bias=True)
  (linear2): Linear(in_features=90, out_features=30, bias=True)
  (l_out): Linear(in_features=30, out_features=2, bias=False)
  (activation_relu): ReLU()
  (activation_sigmoid): Sigmoid()
  (drop_layer_1): Dropout(p=0.4, inplace=False)
  (drop_layer_2): Dropout(p=0.3, inplace=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
# Make a test with some small picture input

localizer.eval()
n = 0

for n in range(len(h_pictures_small)):

  X_SMALL = h_pictures_small[n].astype('float32')
  X_SMALL = X_SMALL.reshape(batch_size, 110, 65, 1)
  X_SMALL = Variable(torch.from_numpy(X_SMALL))


  test_out = localizer(Variable(X_SMALL))
  test_out = test_out.detach().numpy()
  test_out = squeezed_to_coords(test_out)

  X_BIG = h_pictures_large[n]
  X_BIG = X_BIG.reshape(batch_size, 1100, 650, 1)
  network_crop = crop_pictures(X_BIG, test_out)


  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.suptitle('Large pic | Predicted Zoom')
  ax1.imshow(X_BIG.reshape(1100, 650), cmap=plt.get_cmap('gray'))
  ax2.imshow(network_crop.reshape(420, 420), cmap=plt.get_cmap('gray'))
  plt.show()

  path = F"/content/gdrive/My Drive/Deep_Learning/Images_Cropped" 
  cv2.imwrite("Healthy_"+str(n)+".jpg", network_crop[0, :, :, 0]*255)


In [0]:
# Make a test with some small picture input

localizer.eval()
n = 0

for n in range(len(s_pictures_small)):

  X_SMALL = s_pictures_small[n].astype('float32')
  X_SMALL = X_SMALL.reshape(batch_size, 110, 65, 1)
  X_SMALL = Variable(torch.from_numpy(X_SMALL))


  test_out = localizer(Variable(X_SMALL))
  test_out = test_out.detach().numpy()
  test_out = squeezed_to_coords(test_out)

  X_BIG = s_pictures_large[n]
  X_BIG = X_BIG.reshape(batch_size, 1100, 650, 1)
  network_crop = crop_pictures(X_BIG, test_out)

  '''
  fig, (ax1, ax2) = plt.subplots(1, 2)
  fig.suptitle('Large pic | Predicted Zoom')
  ax1.imshow(X_BIG.reshape(1100, 650), cmap=plt.get_cmap('gray'))
  ax2.imshow(network_crop.reshape(420, 420), cmap=plt.get_cmap('gray'))
  plt.show()
  '''
  path = F"/content/gdrive/My Drive/Deep_Learning/Images_Cropped/" 
  cv2.imwrite("Splinter_"+str(n)+".jpg", network_crop[0, :, :, 0]*255)
